In [6]:
import os
import shutil
import random

In [7]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML
import splitfolders
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

In [8]:
IMAGE_SIZE = 256
CHANNELS = 3

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    './output/train',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=32,
    class_mode="sparse"
)

Found 8000 images belonging to 10 classes.


In [10]:
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    './output/val',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=32,
    class_mode="sparse"
)


test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'output/test',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=32,
    class_mode="sparse"
)

Found 1000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [11]:
class_names_arr = train_generator.classes

In [12]:
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 10

In [13]:
import keras
from keras import layers
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import clone_model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model

import keras
import IPython

In [14]:
import keras_tuner
import keras
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Input

In [15]:
def build_model(hp):
    model = keras.models.Sequential()
    
    model.add(Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
    model.add(Conv2D(hp.Int('input_units',
                            min_value=32,
                            max_value=256,
                            step=32), (3, 3), input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))  

    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    for i in range(hp.Int('n_layers', 1, 4)):
        model.add(Conv2D(hp.Int(f'conv_{i}_units',
                                min_value=32,
                                max_value=256,
                                step=32), (3, 3)))
        model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    for i in range(hp.Int('n_layers', 1, 4)):  # adding variation of layers.
        model.add(Conv2D(hp.Int(f'conv_{i}_units',
                                min_value=32,
                                max_value=256,
                                step=32), (3, 3)))
        model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    for i in range(hp.Int('n_connections', 1, 4)):
        model.add(Dense(hp.Choice(f'n_nodes',
                                  values=[128, 256, 512, 1024])))
        model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(Activation("softmax"))

    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",  
                  metrics=["accuracy"])

    return model


In [19]:
tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
 executions_per_trial=3)

In [16]:
print(len(train_generator))

250


In [20]:
tuner.search(
    x=train_generator,  # Pass the generator for training data
    validation_data=validation_generator,
    epochs=2, 
    steps_per_epoch=32,
    validation_steps=32
)

Trial 5 Complete [00h 14m 05s]
val_loss: 0.0

Best val_loss So Far: 0.0
Total elapsed time: 01h 34m 08s


In [ ]:
# Modify the data generators to repeat the data to fix below error

# C:\Users\Osti\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
#   self.gen.throw(typ, value, traceback)

# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True
# ).flow_from_directory(
#     train_dir,
#     target_size=IMAGE_SIZE,
#     batch_size=BATCH_SIZE,
#     class_mode='categorical',
#     shuffle=True  # Shuffle the data for better training
# )

# # Use .repeat() to repeat the training data indefinitely
# train_generator = train_datagen.repeat()

# test_generator = test_datagen.flow_from_directory(
#     test_dir,
#     target_size=IMAGE_SIZE,
#     batch_size=BATCH_SIZE,
#     class_mode='categorical'
# )

# val_generator = test_datagen.flow_from_directory(
#     val_dir,
#     target_size=IMAGE_SIZE,
#     batch_size=BATCH_SIZE,
#     class_mode='categorical'
# )


In [23]:
best_model = tuner.get_best_models(num_models=1)[0]

In [22]:
print(tuner.get_best_models()[0].summary())
print(tuner.get_best_hyperparameters()[0].values)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 96)      │      2,688 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ activation (Activation)         │ (None, 254, 254, 96)      │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 96)      │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 32)      │     27,680 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ activation_1 (Activation)       │ (None, 125, 125, 32)      │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 32)        │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 32)        │      9,248 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ activation_2 (Activation)       │ (None, 60, 60, 32)        │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 32)        │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ flatten (Flatten)               │ (None, 28800)             │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense (Dense)                   │ (None, 512)               │ 14,746,112 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ activation_3 (Activation)       │ (None, 512)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_1 (Dense)                 │ (None, 10)                │      5,130 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ activation_4 (Activation)       │ (None, 10)                │          0 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 14,790,858 (56.42 MB)

 Trainable params: 14,790,858 (56.42 MB)

 Non-trainable params: 0 (0.00 B)

None
{'input_units': 96, 'n_layers': 1, 'conv_0_units': 32, 'n_connections': 1, 'n_nodes': 512}


In [25]:
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 96)      │      2,688 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ activation (Activation)         │ (None, 254, 254, 96)      │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 96)      │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 32)      │     27,680 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ activation_1 (Activation)       │ (None, 125, 125, 32)      │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 32)        │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 32)        │      9,248 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ activation_2 (Activation)       │ (None, 60, 60, 32)        │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 32)        │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ flatten (Flatten)               │ (None, 28800)             │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense (Dense)                   │ (None, 512)               │ 14,746,112 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ activation_3 (Activation)       │ (None, 512)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_1 (Dense)                 │ (None, 10)                │      5,130 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ activation_4 (Activation)       │ (None, 10)                │          0 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 14,790,858 (56.42 MB)

 Trainable params: 14,790,858 (56.42 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
best_model.compile(
    optimizer=tf.optimizers.Adam(), 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [28]:
history = best_model.fit(
    train_generator,
    steps_per_epoch=250,
    batch_size=32,
    validation_data=validation_generator,
    validation_steps=31,
    verbose=1,
    epochs=1,
)

250/250 ━━━━━━━━━━━━━━━━━━━━ 238s 946ms/step - accuracy: 0.4795 - loss: 1.5473 - val_accuracy: 0.7782 - val_loss: 0.7238


In [29]:
from sklearn.metrics import classification_report, confusion_matrix

In [31]:
predictions = best_model.predict(test_generator)
predicted_classes = tf.argmax(predictions, axis=1)
true_classes = test_generator.classes

32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 244ms/step


In [32]:
print("Classification Report:\n", classification_report(true_classes, predicted_classes))
print("\nConfusion Matrix:\n", confusion_matrix(true_classes, predicted_classes))

Classification Report:
               precision    recall  f1-score   support

           0       0.10      0.10      0.10       100
           1       0.12      0.08      0.10       100
           2       0.09      0.06      0.07       100
           3       0.15      0.15      0.15       100
           4       0.10      0.11      0.11       100
           5       0.09      0.10      0.09       100
           6       0.12      0.14      0.13       100
           7       0.07      0.07      0.07       100
           8       0.08      0.08      0.08       100
           9       0.05      0.06      0.05       100

    accuracy                           0.10      1000
   macro avg       0.10      0.10      0.10      1000
weighted avg       0.10      0.10      0.10      1000


Confusion Matrix:
 [[10  8  9  8 13 12  6  9 11 14]
 [ 6  8  4  8  9 16 14 11 14 10]
 [11  5  6  9 13  8 14 15  5 14]
 [14  7  7 15  8 10  6  7 14 12]
 [10  2  9  9 11 15 10 10 10 14]
 [ 9 11  9 10  9 10 10  9 13 10]

In [ ]:
# to assign different weight

In [33]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np


In [39]:
class_labels = np.unique(train_generator.classes)
# class_weights = compute_class_weight('balanced', class_labels, train_generator.classes)
# class_weight_dict = dict(zip(class_labels, class_weights))

class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = class_labels,
                                        y = train_generator.classes                                                 
                                    )
class_weight_dict = dict(zip(class_labels, class_weights))

# Print computed class weights
print("Computed Class Weights:", class_weight_dict)

Computed Class Weights: {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0, 7: 1.0, 8: 1.0, 9: 1.0}


In [ ]:
# Pass class weights to the fit method
history = best_model.fit(
    train_generator,
    steps_per_epoch=250,
    batch_size=32,
    validation_data=validation_generator,
    validation_steps=31,
    verbose=1,
    epochs=1,
    class_weight=class_weight_dict
) 